In [1]:
import pandas as pd, numpy as np
from datetime import datetime
# import yfinance as yf
import tushare as ts
import time, urllib
ts.set_token('2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67')
import glob
import os

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)

In [2]:
ch_etfs_df = pd.read_csv("/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/"+"CH_ETFs.csv")
ch_etfs_df.dropna(inplace = True)
ch_etfs_df['基金规模\n[单位] 元'] = ch_etfs_df['基金规模\n[单位] 元'].apply(lambda x: float(x.replace(",","")))
ch_etfs_df['机构投资者持有份额\n[报告期] 2019中报\n[单位] 份'] = ch_etfs_df['机构投资者持有份额\n[报告期] 2019中报\n[单位] 份'].apply(lambda x: float(x.replace(",","")))
ch_etfs_df.sort_values("基金规模\n[单位] 元", ascending=False, inplace =True)
ch_etfs_df = ch_etfs_df

In [3]:
ch_etfs_df.head(3)

,证券代码,证券简称,投资类型(一级分类),投资类型(二级分类),基金规模\n[单位] 元,机构投资者持有份额\n[报告期] 2019中报\n[单位] 份,机构投资者持有比例\n[报告期] 2019中报\n[单位] %,管理费率\n[单位] %,托管费率\n[单位] %,认购费率\n[收费类型] 前端
152,510500.OF,南方中证500ETF,股票型基金,被动指数型基金,4.349555e+10,6.869517e+09,76.03,0.5,0.1,500万份以上 1000元/笔\n100万份以下 1%\n100~300万份 0.6%\n3...
180,510050.OF,华夏上证50ETF,股票型基金,被动指数型基金,4.166154e+10,1.220778e+10,73.10,0.5,0.1,100万份以下 1%\n100~1000万份 0.8%\n1000万份以上 0.5%\n
162,510300.OF,华泰柏瑞沪深300ETF,股票型基金,被动指数型基金,3.451196e+10,6.917064e+09,73.10,0.5,0.1,100万份以上 1000元/笔\n50万份以下 1%\n50~100万份 0.5%\n


In [4]:
ch_etfs = ch_etfs_df.copy()
ch_etfs['code'] =ch_etfs['证券代码'].apply(lambda x: str(x)[:6])
ch_etfs_ticker = list(ch_etfs['code'].unique())
# Add the sp500 etf
etf_tickers = ['513500']+ch_etfs_ticker

In [2]:
def GetHistoryData(Code, BeginTime, EndTime):
    df = ts.get_k_data(Code, index = False,  start = BeginTime, end = EndTime)
    return df

In [8]:
etf_tickers[:5]

['513500', '510500', '510050', '510300', '510330']

In [5]:
import eventlet
eventlet.monkey_patch()

ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
# ticker_list = etf_tickers
ticker_list = ['511020']
# ticker_list = ['513500',#标普500
#                '501021',#香港中小
#                # 目前黄金ETF基金主要有四只，易方达黄金ETF(159934)、博时黄金159937、国泰黄金ETF（518800）、华安黄金ETF（518880）。
#                '159934','159937','518800','518880'
#                # 300ETF（510300）、创业板ETF（159915）、500ETF（510500）都是指数ETF基金。
#                # 除了这些宽基指数基金外，还有跟踪行业指数的指数ETF，比如金融ETF(510230)、银行ETF（512800）、
#                # 军工ETF（512660）、广发医药（159938）等。
#                '510300','159915','510500','510230','512800','512660','159938']
today = str(datetime.now().date())
start = '2010-01-01'
end = today
count = 1
for ticker in ticker_list:
    if count%200==0:
        print("=======================Sleeping======================")
        time.sleep(60)
    else:
        if not os.path.exists(ch_db_path+ticker+".csv"):
            print("{} is new, start downloading now...".format(ticker))
            try:
                ############ replace get_data with other customized functions #############
                data = GetHistoryData(ticker, start, today)
                ############ replace get_data with other customized functions #############
#                 print(data)
                data['date'] = data['date'].astype(str)
                data['date'] = data['date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                data.sort_values("date", inplace = True)
                data.to_csv(ch_db_path+ticker+".csv", index = False)
                print("{} data file created: {}".format(ticker, end))
            except Exception as e:
                print(e)

        else:
            print("Already have data csv for {}".format(ticker))
            hist_data = pd.read_csv(ch_db_path+ticker+".csv")   
            hist_data['date'] = hist_data['date'].astype(str)
            hist_data['date'] = hist_data['date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
            hist_data.to_csv(ch_db_path+ticker+".csv", index = False)
            hist_data = pd.read_csv(ch_db_path+ticker+".csv")
            try:
                hist_data_last_date = hist_data['date'].values[-1]        
                if today > hist_data_last_date:
                    print("Needs to update, start updating new data for {} now...".format(ticker))
                    update_start = hist_data_last_date
                    update_end = today
                    with eventlet.Timeout(60,False):
                        try:
                            ############ replace get_data with other customized functions #############
                            new_data = GetHistoryData(ticker, update_start, update_end)
                            ############ replace get_data with other customized functions #############
                            
                            new_data['date'] = new_data['date'].astype(str)
                            new_data['date'] = new_data['date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:])
                            new_data.to_csv(ch_db_path+ticker+".csv", mode='a', header=False, index = False)
                            updated_duplicated_df = pd.read_csv(ch_db_path+ticker+".csv")
                            updated_df = updated_duplicated_df.drop_duplicates("date")
                            updated_df.sort_values("date", inplace = True)
                            updated_df.to_csv(ch_db_path+ticker+".csv", index = False)
                            print("New data updated till today for {}!".format(ticker))
                        except Exception as e:
                            print(e)
        #             print("Timed Out: Update Failed!")
                else:
                    print("There's no new data to update for {}.".format(ticker))
            except Exception as e:
                print(e)

    #     print("Data Download/Update for {} is Finished.".format(ticker))
        print("=======================Executed: {}=======================".format(count))
    count+=1
print("【Updated Finished for today!】")

Traceback (most recent call last):
  File "/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/eventlet/hubs/hub.py", line 461, in fire_timers
    timer()
  File "/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/eventlet/hubs/timer.py", line 59, in __call__
    cb(*args, **kw)
  File "/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/eventlet/semaphore.py", line 147, in _do_acquire
    waiter.switch()
greenlet.error: cannot switch to a different thread


Already have data csv for 511020
Needs to update, start updating new data for 511020 now...
Cannot switch to MAINLOOP from MAINLOOP
=======================Executed: 1=======================
【Updated Finished for today!】
